<a href="https://colab.research.google.com/github/satani99/mnist_using_relu/blob/main/mnist_fast_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 4.2 MB 57.3 MB/s 
     |████████████████████████████████| 112 kB 50.7 MB/s 
     |████████████████████████████████| 147 kB 52.1 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
  ERROR: Failed building wheel for lxml
    Running setup.py install for lxml ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-rd7ym7qy/lxml_60578a13d35e4b4799d643c449f8194b/setup.py'"'"'; __file__='"'"'/tmp/pip-install-rd7ym7qy/lxml_60578a13d35e4b4799d643c449f8194b/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-43m5xe5n/install-record.txt --s

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'nikhilsatani'
os.environ['KAGGLE_KEY'] = 'ec88f260eb867087af019a6f279845ff'

In [ ]:
%%bash
mkdir /content/data
cd /content/data
kaggle competitions download -c digit-recognizer 

100%|██████████| 15.3M/15.3M [00:00<00:00, 63.6MB/s]


In [ ]:
!unzip "/content/data/digit-recognizer.zip" -d "/content/data/"

Archive:  /content/data/digit-recognizer.zip
  inflating: /content/data/sample_submission.csv  
  inflating: /content/data/test.csv  
  inflating: /content/data/train.csv  


In [ ]:
import pandas as pd
import torch
from fastai.vision.all import *
import os
from pathlib import Path
train_df = pd.read_csv("/content/data/train.csv")
test_df = pd.read_csv("/content/data/test.csv")

In [ ]:
# data = pd.read_csv("/content/data/data.csv", index_col=0)
submission = pd.read_csv("/content/data/sample_submission.csv")

In [ ]:
Train = Path('/content/train')
Test = Path('/content/test')

In [ ]:
for index in range(10):
  try:
    os.makedirs(Train/str(index))
  except:
    pass

In [ ]:
sorted(os.listdir(Train))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
try:
  os.makedirs(Test)
except:
  pass

In [ ]:
os.listdir(Test)

[]

In [ ]:
from PIL import Image

def saveDigit(digit, filepath):
  digit = digit.reshape(28, 28)
  digit = digit.astype(np.uint8)

  img = Image.fromarray(digit)
  img.save(filepath)

In [ ]:
for index, column in train_df.iterrows():
  label, digit = column[0], column[1:]
  digit = digit.values
  folder = Train/str(label)
  filename = f"{index}.jpg"
  filepath = folder/filename
  saveDigit(digit, filepath)

In [ ]:
for index, digit in test_df.iterrows():
  folder = Test
  filename = f"{index}.jpg"
  filepath = folder/filename
  digit = digit.values
  saveDigit(digit, filepath)

In [ ]:
digits = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y = parent_label,
    item_tfms=Resize(128)
)

In [ ]:
dls = digits.dataloaders(Train)

In [ ]:
learn = vision_learner(dls, resnet152, metrics=accuracy)
learn.fit_one_cycle(10)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:136: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.149142,0.093019,0.973095,03:21
1,0.054742,0.062708,0.984643,03:21
2,0.054846,0.045196,0.987262,03:21
3,0.035367,0.040975,0.989643,03:20
4,0.027553,0.033094,0.991667,03:20
5,0.010892,0.028734,0.993452,03:20
6,0.005393,0.028295,0.993452,03:20
7,0.003775,0.025423,0.994881,03:20
8,0.001241,0.026566,0.994405,03:20
9,0.001615,0.025982,0.995000,03:21


In [ ]:
learn.predict('/content/test/10107.jpg')

('0',
 TensorBase(0),
 TensorBase([1.0000e+00, 6.9501e-10, 1.4057e-10, 2.0871e-10, 4.7496e-11,
             8.9108e-11, 3.7531e-09, 7.3168e-10, 2.6178e-10, 2.7758e-09]))

In [ ]:
test_dl = dls.test_dl(get_image_files(Test))

In [ ]:
print(len(test_dl.items))

28000


In [ ]:
datasetIndex = []
num = len(test_dl.items)

for i in range(num):
    datasetIndex.append(str(test_dl.items[i]).split('/')[-1][:-4])

In [ ]:
print(len(datasetIndex))
print(datasetIndex)

28000
['8514', '8181', '14672', '17091', '7870', '23530', '2633', '16872', '14862', '4811', '23013', '16308', '2146', '14319', '4871', '4699', '8637', '5200', '26730', '9584', '17674', '3914', '18989', '1782', '3053', '20734', '14479', '15379', '12258', '15793', '13447', '2793', '2680', '3997', '25057', '11137', '18553', '16145', '18380', '22889', '24630', '18191', '16192', '16340', '23006', '627', '4792', '20655', '1988', '5803', '22654', '8688', '16082', '2097', '23796', '8167', '18481', '1619', '2571', '21092', '26672', '9052', '22452', '7186', '3213', '24305', '20799', '26213', '5438', '24455', '20105', '16353', '8451', '26783', '12940', '6115', '13123', '12414', '4341', '3102', '18640', '11740', '291', '13834', '12923', '18472', '27628', '22278', '7649', '23327', '18156', '4826', '634', '4630', '4035', '22218', '2716', '11703', '592', '8336', '15575', '23814', '22987', '20534', '17166', '16215', '15362', '17874', '4934', '22914', '21466', '5845', '1374', '10885', '18659', '12787',

In [ ]:
datasetIndex = [int(i) for i in datasetIndex]

In [ ]:
print(datasetIndex)
# print(sorted(datasetIndex))

[8514, 8181, 14672, 17091, 7870, 23530, 2633, 16872, 14862, 4811, 23013, 16308, 2146, 14319, 4871, 4699, 8637, 5200, 26730, 9584, 17674, 3914, 18989, 1782, 3053, 20734, 14479, 15379, 12258, 15793, 13447, 2793, 2680, 3997, 25057, 11137, 18553, 16145, 18380, 22889, 24630, 18191, 16192, 16340, 23006, 627, 4792, 20655, 1988, 5803, 22654, 8688, 16082, 2097, 23796, 8167, 18481, 1619, 2571, 21092, 26672, 9052, 22452, 7186, 3213, 24305, 20799, 26213, 5438, 24455, 20105, 16353, 8451, 26783, 12940, 6115, 13123, 12414, 4341, 3102, 18640, 11740, 291, 13834, 12923, 18472, 27628, 22278, 7649, 23327, 18156, 4826, 634, 4630, 4035, 22218, 2716, 11703, 592, 8336, 15575, 23814, 22987, 20534, 17166, 16215, 15362, 17874, 4934, 22914, 21466, 5845, 1374, 10885, 18659, 12787, 11438, 22043, 24478, 21770, 17416, 9882, 13014, 25644, 25581, 9306, 404, 43, 5298, 13588, 25771, 3290, 22328, 3775, 22349, 27790, 9861, 13378, 16465, 13747, 99, 11521, 22725, 26994, 25458, 8907, 23113, 13261, 25096, 1515, 15772, 14081, 1

In [ ]:
prediction, _ = learn.get_preds(dl=test_dl)

In [ ]:
prediction = np.argmax(prediction, axis=1)

In [ ]:
prediction


TensorBase([4, 5, 8,  ..., 1, 4, 1])

In [ ]:
submission['ImageId'] = datasetIndex
submission['Label'] = prediction
submission = submission.sort_values(by=['ImageId'])
submission.to_csv('submission.csv', index=False)

In [ ]:
sub = pd.read_csv('submission.csv')
sub.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [ ]:
from google.colab import files
files.download('submission.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>